# Yelp Data Challenge - Data Preprocessing

BitTiger DS501

Jun 2017

## Dataset Introduction

[Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge)

The Challenge Dataset:

    4.1M reviews and 947K tips by 1M users for 144K businesses
    1.1M business attributes, e.g., hours, parking availability, ambience.
    Aggregated check-ins over time for each of the 125K businesses
    200,000 pictures from the included businesses

Cities:

    U.K.: Edinburgh
    Germany: Karlsruhe
    Canada: Montreal and Waterloo
    U.S.: Pittsburgh, Charlotte, Urbana-Champaign, Phoenix, Las Vegas, Madison, Cleveland

Files:

    yelp_academic_dataset_business.json
    yelp_academic_dataset_checkin.json
    yelp_academic_dataset_review.json
    yelp_academic_dataset_tip.json
    yelp_academic_dataset_user.json

Notes on the Dataset

    Each file is composed of a single object type, one json-object per-line.
    Take a look at some examples to get you started: https://github.com/Yelp/dataset-examples.



## Read data from file and load to Pandas DataFrame

**Warning**: Loading all the 1.8 GB data into Pandas at a time takes long time and a lot of memory!

In [ ]:
import json
import pandas as pd

In [ ]:
file_business, file_checkin, file_review, file_tip, file_user = [
    'yelp_dataset_challenge_round9/yelp_academic_dataset_business.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_checkin.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_review.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_tip.json',
    'yelp_dataset_challenge_round9/yelp_academic_dataset_user.json'
]

#### Business Data

In [ ]:
with open(file_business) as f:
    df_business = pd.DataFrame(json.loads(line) for line in f)

In [ ]:
df_business.head(2)

In [ ]:
df_business.info()

#### Checkin Data

In [ ]:
# with open(file_checkin) as f:
#     df_checkin = pd.DataFrame(json.loads(line) for line in f)
# df_checkin.head(2)

#### Review Data

In [ ]:
# with open(file_review) as f:
#     df_review = pd.DataFrame(json.loads(line) for line in f)
# df_review.head(2)

#### Tip Data

In [ ]:
# with open(file_tip) as f:
#     df_tip = pd.DataFrame(json.loads(line) for line in f)
# df_tip.head(2)

#### User Data

In [ ]:
# with open(file_user) as f:
#     df_user = pd.DataFrame(json.loads(line) for line in f)
# df_user.head(2)

## Filter data by city and category

#### Create filters/masks

* create filters that selects business 
    * that are located in "Las Vegas"
    * that contains "Restaurants" in their category (You may need to filter null categories first)

In [ ]:
# Create Pandas DataFrame filters
pass

In [ ]:
# Create filtered DataFrame, and name it df_filtered
pass

#### Keep relevant columns

* only keep some useful columns
    * business_id
    * name
    * categories
    * stars

In [ ]:
selected_features = [u'business_id', u'name', u'categories', u'stars']

In [ ]:
# Make a DataFrame that contains only the abovementioned columns, and name it as df_selected_business
pass

In [ ]:
# Rename the column name "stars" to "avg_stars" to avoid naming conflicts with review dataset
pass

In [ ]:
# Inspect your DataFrame
pass

#### Save results to csv files

In [ ]:
# Save to ./data/selected_business.csv for your next task
pass

In [ ]:
# Try reload the csv file to check if everything works fine
pass

### Use the "business_id" column to filter review data

* We want to make a DataFrame that contain and only contain the reviews about the business entities we just obtained

#### Load review dataset

In [ ]:
with open(file_review) as f:
    df_review = pd.DataFrame(json.loads(line) for line in f)
df_review.head(2)

#### Prepare dataframes to be joined, - on business_id

In [ ]:
# Prepare the business dataframe and set index to column "business_id", and name it as df_left
pass

In [ ]:
# Prepare the review dataframe and set index to column "business_id", and name it as df_right
pass

#### Join! and reset index

In [ ]:
# Join df_left and df_right. What type of join?
pass

In [ ]:
# You may want to reset the index 
pass

#### We further filter data by date, e.g. keep comments from last 2 years

* Otherwise your laptop may crush on memory when running machine learning algorithms
* Purposefully ignoring the reviews made too long time ago

In [ ]:
# Make a filter that selects date after 2015-01-20
pass

In [ ]:
# Filter the joined DataFrame and name it as df_final
pass

#### Take a glance at the final dataset

* Do more EDA here as you like!

In [ ]:
import matplotlib.pyplot as plt

% matplotlib inline

In [ ]:
# e.g. calculate counts of reviews per business entity, and plot it
pass

## Save your preprocessed dataset to csv file

* Respect your laptop's hard work! You don't want to make it run everything again.

In [ ]:
# Save to ./data/last_2_years_restaurant_reviews.csv for your next task
pass